In [1]:
from fastprogress import progress_bar
import torch
import matplotlib as mpl
import fastcore.all as fc
from datasets import load_dataset
from miniai.datasets import inplace, DataLoaders

/scratch/users/carles/mambaforge/envs/mini-diffusers/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
from miniai.utils import set_seed

In [3]:
torch.set_printoptions(precision=5, linewidth=140, sci_mode=False)
torch.manual_seed(1)
mpl.rcParams['image.cmap'] = 'gray_r'
mpl.rcParams['figure.dpi'] = 70

import logging
logging.disable(logging.WARNING)

set_seed(42)
if fc.defaults.cpus>8: fc.defaults.cpus=8

In [4]:
xl,yl = 'image','label'
name = "fashion_mnist"
n_steps = 1000
bs = 512
dsd = load_dataset(name)

In [5]:
from miniai.diffusion import dl_ddpm
import torch.nn.functional as F
import torchvision.transforms.functional as TF

@inplace
def transformi(b): 
    b[xl] = [F.pad(TF.to_tensor(o), (2,2,2,2))*2-1 for o in b[xl]]

In [6]:
tds = dsd.with_transform(transformi)
dls = DataLoaders(dl_ddpm(tds['train']), dl_ddpm(tds['test']))

# Train

In [7]:
from functools import partial
from torch.optim import lr_scheduler
from torch import optim
from miniai.learner import DeviceCB, ProgressCB, MetricsCB
from miniai.sgd import BatchSchedCB
from miniai.accel import MixedPrecision
from miniai.unet import EmbUNetModel
from miniai.learner import Learner
import torch.nn as nn

In [8]:
lr = 1e-2
epochs = 25
opt_func = partial(optim.Adam, eps=1e-5)
tmax = epochs * len(dls.train)

sched = partial(lr_scheduler.OneCycleLR, max_lr=lr, total_steps=tmax)

cbs = [DeviceCB(), ProgressCB(plot=True), MetricsCB(), BatchSchedCB(sched), MixedPrecision()]

model = EmbUNetModel(in_channels=1, out_channels=1, nfs=(32,64,128,256), num_layers=2)

learn = Learner(model, dls, nn.MSELoss(), lr=lr, cbs=cbs, opt_func=opt_func)

In [9]:
learn.fit(epochs)

ValueError: Expected more than 1 value per channel when training, got input size torch.Size([1, 32])